# Testing de Datos

Este notebook demuestra cómo escribir tests efectivos para transformaciones de datos.

**Referencia:** [Testing de Datos](../calidad/validaciones/testing-de-datos.md)

**Objetivos:**
- Escribir tests unitarios para transformaciones
- Tests de integridad
- Tests de calidad
- Tests de reglas de negocio

## 1. Importar librerías

In [ ]:
import pandas as pd
import numpy as np

# Para testing (pytest)
try:
    import pytest
    print("✅ pytest disponible")
except ImportError:
    print("💡 Para instalar: pip install pytest")

print("✅ Librerías importadas")

## 2. Funciones a testear

In [ ]:
def calcular_total(precio, cantidad):
    """Calcula el total de una venta."""
    return precio * cantidad

def limpiar_nombres(df):
    """Limpia nombres en un DataFrame."""
    df = df.copy()
    if 'nombre' in df.columns:
        df['nombre'] = df['nombre'].str.strip().str.title()
    return df

def transformar_ventas(df):
    """Transforma datos de ventas."""
    df = df.copy()
    df['total'] = df['precio'] * df['cantidad']
    df = df[df['total'] > 0]  # Filtrar totales negativos
    return df

print("✅ Funciones definidas")

## 3. Tests unitarios

In [ ]:
# Test 1: calcular_total
def test_calcular_total():
    """Test de función calcular_total."""
    assert calcular_total(10, 2) == 20
    assert calcular_total(5.5, 3) == 16.5
    assert calcular_total(0, 10) == 0
    print("✅ test_calcular_total: PASÓ")

test_calcular_total()

In [ ]:
# Test 2: limpiar_nombres
def test_limpiar_nombres():
    """Test de limpieza de nombres."""
    df = pd.DataFrame({
        'nombre': ['  juan  ', 'MARÍA', 'carlos']
    })
    
    resultado = limpiar_nombres(df)
    
    assert resultado['nombre'].tolist() == ['Juan', 'María', 'Carlos']
    print("✅ test_limpiar_nombres: PASÓ")

test_limpiar_nombres()

## 4. Tests de integridad

In [ ]:
# Test: Sin pérdida de filas críticas
def test_sin_perdida_filas_criticas():
    """Verifica que no se pierdan filas críticas."""
    df_entrada = pd.DataFrame({
        'id': [1, 2, 3, 4, 5],
        'es_critico': [True, False, True, False, False],
        'precio': [10, 20, 30, 40, 50]
    })
    
    # Simular transformación que podría eliminar filas
    # IMPORTANTE: Las filas críticas deben preservarse incluso si no cumplen el filtro
    df_salida = df_entrada[
        (df_entrada['precio'] > 15) | (df_entrada['es_critico'] == True)
    ].copy()
    
    # Verificar que filas críticas no se perdieron
    ids_criticos = df_entrada[df_entrada['es_critico'] == True]['id'].tolist()
    ids_en_salida = df_salida['id'].tolist()
    
    for id_critico in ids_criticos:
        assert id_critico in ids_en_salida, f"ID crítico {id_critico} perdido"
    
    print("✅ test_sin_perdida_filas_criticas: PASÓ")
    print(f"   IDs críticos preservados: {ids_criticos}")
    print(f"   IDs en salida: {ids_en_salida}")

test_sin_perdida_filas_criticas()

In [ ]:
# Test: Suma conservada
def test_suma_conservada():
    """Verifica que sumas se conserven."""
    df_entrada = pd.DataFrame({
        'precio': [10, 20, 30, 40, 50]
    })
    
    df_salida = transformar_ventas(pd.DataFrame({
        'precio': [10, 20, 30, 40, 50],
        'cantidad': [1, 1, 1, 1, 1]
    }))
    
    total_entrada = df_entrada['precio'].sum()
    total_salida = df_salida['total'].sum()
    
    # Permitir pequeña diferencia por redondeo
    assert abs(total_entrada - total_salida) < 0.01, "Suma no se conserva"
    print("✅ test_suma_conservada: PASÓ")

test_suma_conservada()

## 5. Tests de calidad

In [ ]:
# Test: Completitud mínima
def test_completitud_minima(df, umbral=0.95):
    """Verifica completitud mínima."""
    for col in df.columns:
        completitud = (1 - df[col].isnull().sum() / len(df)) * 100
        assert completitud >= umbral * 100, \
            f"Completitud de {col} es {completitud}%, menor que {umbral*100}%"

# Ejemplo
df_ejemplo = pd.DataFrame({
    'col1': [1, 2, 3, 4, 5],
    'col2': [1, 2, None, 4, 5],  # 80% completitud
    'col3': [1, 2, 3, 4, 5]
})

try:
    test_completitud_minima(df_ejemplo, umbral=0.95)
    print("✅ test_completitud_minima: PASÓ")
except AssertionError as e:
    print(f"⚠️ test_completitud_minima: FALLÓ (esperado para col2)")
    print(f"   {str(e)[:100]}")

In [ ]:
# Test: Sin duplicados
def test_sin_duplicados(df, columnas_unicas):
    """Verifica que no haya duplicados."""
    for col in columnas_unicas:
        if col in df.columns:
            duplicados = df[col].duplicated().sum()
            assert duplicados == 0, f"Duplicados encontrados en {col}: {duplicados}"

# Ejemplo
df_ejemplo = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'email': ['a@email.com', 'b@email.com', 'c@email.com', 'd@email.com', 'e@email.com']
})

test_sin_duplicados(df_ejemplo, ['id', 'email'])
print("✅ test_sin_duplicados: PASÓ")

## 6. Tests de reglas de negocio

In [ ]:
# Test: Regla de negocio - descuento máximo
def test_descuento_maximo(df):
    """Verifica que descuentos no excedan 50%."""
    if 'descuento' in df.columns:
        descuento_max = df['descuento'].max()
        assert descuento_max <= 0.5, f"Descuento máximo {descuento_max} excede 50%"

# Ejemplo
df_ventas = pd.DataFrame({
    'producto': ['A', 'B', 'C'],
    'precio': [100, 200, 300],
    'descuento': [0.1, 0.2, 0.3]  # Todos < 50%
})

test_descuento_maximo(df_ventas)
print("✅ test_descuento_maximo: PASÓ")

## 7. Resumen de tests

In [ ]:
print("=" * 60)
print("RESUMEN DE TESTS")
print("=" * 60)
print("\n✅ Tests unitarios: Funciones individuales")
print("✅ Tests de integridad: Sin pérdida de datos")
print("✅ Tests de calidad: Métricas de calidad")
print("✅ Tests de reglas de negocio: Validaciones de negocio")
print("\n" + "=" * 60)
print("\n💡 Para ejecutar con pytest:")
print("   pytest nombre_archivo.py -v")
print("=" * 60)